In [1]:
import pandas as pd
from models import CARCalculator

In [4]:
# 把证券代码添加进来
df_raw_CAR = pd.read_excel("./data/processed/股份回购与分红合并_添加Treat_Post.xlsx", skiprows=[1], dtype={"SCode_回购":str})
df_raw_raw_CAR = pd.read_excel("./data/raw/股份回购.xlsx", skiprows=[1], dtype={"SCode":str})
df_industry = pd.read_excel("./data/raw/公司行业分类.xlsx", dtype={"股票代码":str})

print(f"原始数据行数: {len(df_raw_CAR)}")
print(f"股份回购数据行数: {len(df_raw_raw_CAR)}")
print(f"行业分类数据行数: {len(df_industry)}")

# 检查重复
print("\n检查重复键:")
print(f"df_raw_raw_CAR 中 SCode 重复数: {df_raw_raw_CAR['SCode'].duplicated().sum()}")
print(f"df_industry 中 股票代码 重复数: {df_industry['股票代码'].duplicated().sum()}")

# 重新开始，避免重复合并
df_CAR = df_raw_CAR.copy()

# 第一次合并：添加 BPAmtDy（去重后合并）
# 如果有重复，保留第一条
df_raw_raw_CAR_unique = df_raw_raw_CAR.drop_duplicates(subset=['SCode'], keep='first')
print(f"\n去重后的股份回购数据: {len(df_raw_raw_CAR_unique)} 行")

df_CAR = pd.merge(df_CAR, df_raw_raw_CAR_unique[['SCode', 'BPAmtDy']], 
                  left_on='SCode_回购', right_on='SCode', how='left').drop(columns=['SCode'])
print(f"合并 BPAmtDy 后: {len(df_CAR)} 行")

# 第二次合并：添加证券代码（去重后合并）
df_industry_unique = df_industry.drop_duplicates(subset=['股票代码'], keep='first')
print(f"去重后的行业数据: {len(df_industry_unique)} 行")

df_CAR = pd.merge(df_CAR, df_industry_unique[['证券代码','股票代码']], 
                  left_on='SCode_回购', right_on='股票代码', how='left').drop(columns=['股票代码'])
print(f"合并证券代码后: {len(df_CAR)} 行")

# 验证最终数据
if len(df_CAR) != len(df_raw_CAR):
    print(f"\n⚠️ 警告: 最终行数 ({len(df_CAR)}) 与原始行数 ({len(df_raw_CAR)}) 不一致！")
else:
    print(f"\n✓ 合并成功，行数一致: {len(df_CAR)} 行")

# 检查数据
print(f"\n数据预览:")
df_CAR.head(5)

原始数据行数: 11464
股份回购数据行数: 11464
行业分类数据行数: 5177

检查重复键:
df_raw_raw_CAR 中 SCode 重复数: 8512
df_industry 中 股票代码 重复数: 1

去重后的股份回购数据: 2952 行
合并 BPAmtDy 后: 11464 行
去重后的行业数据: 5176 行
合并证券代码后: 11464 行

✓ 合并成功，行数一致: 11464 行

数据预览:


,SCode_回购,Year,SoAmDAD,RepMaNum,RepMiNum,RepType,RepProc,MaxPrc,MinPrc,AvePrc,...,EqBenmkDt,AnnDt,RecordDt,DRDt,DPRDt,BnsShrLDt,Treat,Post,BPAmtDy,证券代码
0,000002,2015,2015-09-01,12480299.0,12480299.0,普通回购,实施完成,1.000000e+10,NaN,12.83,...,2016-07-28,2016-07-22,2016-07-28,2016-07-29,2016-07-29,NaN,1,0,2015-07-06,000002.SZ
1,000002,2022,NaN,136836344.0,109469075.0,普通回购,实施中,2.500000e+09,2.000000e+09,17.94,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2015-07-06,000002.SZ
2,000004,2021,NaN,9049599.0,9049599.0,普通回购,完成实施,1.000000e+00,1.000000e+00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2021-05-27,000004.SZ
3,000004,2022,2022-05-21,23155018.0,23155018.0,普通回购,股东大会通过,1.000000e+00,1.000000e+00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2021-05-27,000004.SZ
4,000004,2023,2023-05-20,939474.0,939474.0,普通回购,实施中,1.000000e+00,1.000000e+00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2021-05-27,000004.SZ


In [ ]:
# 随机抽样测试 CAR 计算
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 随机抽样 100 行
sample_size = 100
df_CAR_sample = df_CAR.sample(n=min(sample_size, len(df_CAR)), random_state=42).copy()
print(f"抽样测试: {len(df_CAR_sample)} 行")

# 存储错误信息
error_records = []

def calculate_single_car(row):
    '''
    单个 CAR 计算函数
    '''
    try:
        calculator = CARCalculator(
            stock_code=row['证券代码'],
            announcement_date=row['BPAmtDy'],
        )
        car_result = calculator.predict_car()
        return car_result["CAR"]
    
    except Exception as e:
        # 记录错误信息
        error_info = {
            '行号': row.name,
            '证券代码': row.get('证券代码', 'N/A'),
            'SCode_回购': row.get('SCode_回购', 'N/A'),
            'BPAmtDy': str(row.get('BPAmtDy', 'N/A')),
            '错误类型': type(e).__name__,
            '错误信息': str(e)[:200]
        }
        error_records.append(error_info)
        return None

# 计算 CAR（使用 tqdm 显示进度）
print("\n开始计算...")
tqdm.pandas(desc="计算进度")
df_CAR_sample['CAR'] = df_CAR_sample.progress_apply(calculate_single_car, axis=1)

# 创建错误表
df_errors = pd.DataFrame(error_records)

# 统计信息
print("\n" + "="*60)
print("抽样测试统计")
print("="*60)
print(f"测试行数: {len(df_CAR_sample)}")
print(f"成功计算 CAR: {df_CAR_sample['CAR'].notna().sum()}")
print(f"失败行数: {df_CAR_sample['CAR'].isna().sum()}")
print(f"成功率: {df_CAR_sample['CAR'].notna().sum() / len(df_CAR_sample) * 100:.2f}%")

if len(df_errors) > 0:
    print(f"\n错误类型分布:")
    print(df_errors['错误类型'].value_counts())
    print(f"\n所有错误详情:")
    print(df_errors)
else:
    print("\n✓ 没有错误！")

print("\n" + "="*60)
print(f"CAR 统计信息:")
print(df_CAR_sample['CAR'].describe())
print("="*60)

# 显示成功的样本
print(f"\n成功计算的样本预览:")
print(df_CAR_sample[df_CAR_sample['CAR'].notna()][['证券代码', 'SCode_回购', 'BPAmtDy', 'CAR']].head(10))

准备计算 CAR...
总行数: 11464

开始计算...


计算进度:   0%|          | 27/11464 [00:54<6:21:25,  2.00s/it]



KeyboardInterrupt: 

In [ ]:
# 保存结果
# 1. 保存添加 CAR 后的完整表
output_file_car = "./data/processed/股份回购与分红_CAR结果.xlsx"
df_CAR.to_excel(output_file_car, index=False)
print(f"✓ CAR 结果已保存到: {output_file_car}")

# 2. 保存错误表
if len(df_errors) > 0:
    output_file_errors = "./data/processed/CAR计算错误记录.xlsx"
    df_errors.to_excel(output_file_errors, index=False)
    print(f"✓ 错误记录已保存到: {output_file_errors}")
else:
    print("✓ 没有错误需要保存")

# 3. 显示结果预览
print("\n成功计算的数据预览:")
print(df_CAR[df_CAR['CAR'].notna()][['证券代码', 'SCode_回购', 'BPAmtDy', 'CAR']].head(10))